In [12]:
import os, sys
import pandas as pd
import matplotlib.pyplot as plt

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from utils.data_processing import clean_dir, save_audio_segments_to_disk, load_audio_files, calculate_class_totals, extract_balanced_segments

In [ ]:
CLASS_CAP = 100
SEG_LENGTH = 5.0  # seconds
THRESH = 0.75  # energy threshold

In [ ]:
def extract_audio_segments(audio_files, sr=32000, segment_sec=SEG_LENGTH, cap_per_class=CLASS_CAP):
    """
    Extract high-energy audio segments from audio files with balanced sampling per class.
    
    Args:
        segments_df (pd.DataFrame): DataFrame with 'filename', 'class_id', and 'author' columns
        segments_dir (str): Directory containing original audio .wav files
        sr (int): Target sampling rate
        segment_sec (float): Duration of each segment in seconds
        thresh_factor (float): RMS threshold factor for segment filtering
        cap_per_class (int): Maximum segments per class
        
    Returns:
        list: List of segment dictionaries with metadata including author
    """
    
    # Load and prepare audio files

    if not audio_files:
        print("No valid audio files loaded!")
        return []
    
    print(f"Loaded {len(audio_files)} audio files successfully.")
    
    # Calculate total segments per class for metadata
    class_total_segments = calculate_class_totals(audio_files)
    
    # Extract segments with balanced sampling
    segments = extract_balanced_segments(audio_files, cap_per_class, segment_sec, sr, class_total_segments)
    
    print(f"Extracted {len(segments)} total segments")
    return segments

In [15]:
# Define Paths
audios_dir = os.path.join('..', 'database', 'audio', 'dev')
dev_df = pd.read_csv(os.path.join('..', 'database', 'meta', 'dev_data.csv'))

# Output directories for extracted segments
segments_output_dir = os.path.join('..', 'database', 'audio_segments')
segments_csv_path = os.path.join('..', 'database', 'meta', 'audio_segments.csv')

# Get the files
audio_files = load_audio_files(dev_df, audios_dir, sr=32000, segment_sec=SEG_LENGTH, thresh_factor=THRESH)

In [28]:
import importlib
importlib.reload(sys.modules['utils.data_processing'])
from utils.data_processing import clean_dir, save_audio_segments_to_disk, load_audio_files, calculate_class_totals, extract_balanced_segments

In [29]:
segments = extract_audio_segments(audio_files, sr=32000, segment_sec=SEG_LENGTH, cap_per_class=CLASS_CAP)

print(f"Extracted {len(segments)} segments total")

Loaded 876 audio files successfully.


Extracting segments: 100%|██████████| 876/876 [00:00<00:00, 3462.43it/s]

Extracted 3101 total segments
Extracted 3101 segments total


In [ ]:
# Clan & Save
clean_dir(segments_output_dir)

print("Saving audio segments to disk...")
segments_df = save_audio_segments_to_disk(segments, segments_output_dir)
segments_df.to_csv(segments_csv_path, index=False)

print(f"Saved {len(segments_df)} audio segments to {segments_output_dir}")
print(f"Metadata saved to {segments_csv_path}")

In [ ]:
# Check for repeated values in segments_df 'filename' column
filename_counts = segments_df['filename'].value_counts()
duplicates = filename_counts[filename_counts > 1]

if len(duplicates) > 0:
    print(f"Found {len(duplicates)} repeated filenames:")
    print(duplicates)
else:
    print("No repeated filenames found")

print(f"\nTotal unique filenames: {segments_df['filename'].nunique()}")
print(f"Total rows: {len(segments_df)}")

In [ ]:
# Create histogram showing amount of samples for each class_id
class_counts = segments_df['class_id'].value_counts().sort_index()

plt.figure(figsize=(10, 6))
plt.bar(class_counts.index, class_counts.values)
plt.xlabel('Class ID')
plt.ylabel('Number of Samples')
plt.title('Distribution of Audio Segments by Class ID')
plt.xticks(class_counts.index)
plt.grid(axis='y', alpha=0.3)

# Add value labels on top of bars
for i, v in enumerate(class_counts.values):
    plt.text(class_counts.index[i], v + 0.5, str(v), ha='center', va='bottom')

plt.tight_layout()
plt.show()

print(f"Class distribution:")
for class_id, count in class_counts.items():
    print(f"Class {class_id}: {count} samples")